### Instalando pacotes

In [1]:

!pip install openpyxl
!pip install pandas
!pip install numpy

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.9 kB 262.6 kB/s eta 0:00:01
   ------ -------------------------------- 41.0/250.9 kB 281.8 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.9 kB 479.1 kB/s eta 0:00:01
   -------------------------- ----------- 174.1/250.9 kB 751.6 kB/s eta 0:00:01
   -------------------------------------- 250.9/250.9 kB 908.2 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\pedro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import openpyxl

# Avaliação 03
Grupo 5:
*   Allane
*   Item da lista



### Instanciando os grafos
Estamos usando DataFrames da biblioteca pandas para representar as matrizes de adjacencia dos grafos.

In [3]:
matriz_problema_km = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Km")
matriz_problema_min = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Min")
cidades = pd.read_excel("./PCV__Matriz_do_problema.xlsx", sheet_name="Cidades")

### Estrutura da Busca Local

In [6]:
# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        custo += grafo[v1][v2 - 1]
    return custo

# O argumento 'heuristica' eh o algoritmo da busca local que sera realizada (shift, swap etc).
# ele deve retornar uma vizinhanca - a lista de solucoes geradas
def busca_local_primeira_melhoria(solucao, custo_original, heuristica, grafo):
    custo_atual = custo_original
    solucao_atual = solucao
    melhoria = True
    while melhoria:
        melhoria = False
        for nova_solucao in heuristica(solucao_atual):
            novo_custo = calcular_custo(nova_solucao, grafo)
            if novo_custo < custo_atual:
                solucao_atual = nova_solucao
                custo_atual = novo_custo
                melhoria = True
                break
    return solucao_atual, custo_atual


### Heurísticas

In [ ]:
# As heuristicas funcionam como geradores (yield) que retornam uma vizinhanca
# Elas utilizam uma copia da solucao original para nao modificar a solucao passada como argumento
# Todas as heuristicas preservam o primeiro e o ultimo vertice (de origem e destino), através do range iniciado em 1 e terminado em len(solucao)-1

def shift(solucao : list):
    for idx_vert_shift in range(1, len(solucao) - 1):
        for idx_destino_shift in range(2, len(solucao) - 1):
            solucao_cp = solucao[:] # Cria uma copia da solucao original
            vert_shift = solucao_cp.pop(idx_vert_shift) # Remove o vertice da posicao original
            solucao_cp.insert(idx_destino_shift, vert_shift) # Insere o vertice na nova posicao
            yield solucao_cp # Retorna a nova solucao gerada

def swap(solucao : list):
    for idx_vert_swap in range(1, len(solucao) - 1):
        for idx_destino_swap in range(2, len(solucao) - 1):
            solucao_cp = solucao[:]
            vert_swap = solucao_cp[idx_vert_swap] # Armazena o vertice da posicao original
            solucao_cp[idx_vert_swap] = solucao_cp[idx_destino_swap] # Coloca o vertice destino na posicao original
            solucao_cp[idx_destino_swap] = vert_swap # Coloca o vertice original na posicao destino
            # print("Solucao do swap : ", solucao_cp)
            yield solucao_cp

def inversao(solucao):
    for idx_vert_inicio in range(1, len(solucao) - 1):
        for idx_destino in range(2, len(solucao) - 1):
            solucao_cp = solucao[:]
            sublista_invertida = solucao_cp[idx_vert_inicio:idx_destino] # Extrai a sublista a ser invertida
            sublista_invertida.reverse() # Inverte a sublista
            yield solucao_cp[0:idx_vert_inicio] + sublista_invertida + solucao_cp[idx_destino:] # Reconstrói a solução com a sublista invertida


## Busca por vizinho mais próximo

In [8]:
# Estamos considerando o label numerico
def vizinho_mais_proximo(grafo : pd.DataFrame, vertice_inicial : int | str):
    # Criando conjunto de vertices nao visitados
    # o conjunto contem os indices referentes ao vertice na matriz problema
    vertices_nao_visitados = set(grafo.index)
    percurso = [vertice_inicial]
    vertices_nao_visitados.remove(vertice_inicial - 1)
    dist_total = 0
    v_atual = vertice_inicial
    while len(vertices_nao_visitados) > 0:
        # Obtem o label (numero) do vertice com menor custo
        idx_vizinho_mais_proximo = grafo[v_atual].loc[list(vertices_nao_visitados)].idxmin(skipna=True)
        prox = idx_vizinho_mais_proximo + 1
        dist_total += grafo[v_atual][idx_vizinho_mais_proximo]
        percurso.append(int(prox))
        v_atual = prox
        vertices_nao_visitados.remove(idx_vizinho_mais_proximo)
    dist_total += grafo[v_atual][vertice_inicial -1]
    percurso.append(vertice_inicial)
    return percurso, dist_total


In [59]:
resultados_vmp = [None] * 12
resultados_vmp_busca_local = [None] * 12

## Aplicando nos problemas

In [12]:
def aplicar_problema(grafo, inicial, heuristica_construtiva, heuristicas_busca_local, out = None):
    caminho, custo = heuristica_construtiva(grafo, inicial)
    melhor_custo = custo
    melhor_caminho = caminho
    heuristica_melhor_solucao = None
    out = True if out is None else out
    if out:
        print("Aplicando Heuristica Construtiva: ", heuristica_construtiva.__name__)
        print(f"Caminho: {caminho}")
        print(f"Custo: {custo}")
        print("Aplicando busca(s) local(is):")
    for h in heuristicas_busca_local:
        solucao_local, custo_local = busca_local_primeira_melhoria(melhor_caminho, melhor_custo, h, grafo)
        if out:
            print(f"Heuristica : {h.__name__}")
            print("Tipo : Primeira Melhoria")
            print(f"Melhor caminho pos busca local: {solucao_local}")
            print(f"Melhor custo pos busca local: {custo_local}")
        if custo_local < melhor_custo:
            melhor_caminho = solucao_local
            melhor_custo = custo_local
            heuristica_melhor_solucao = h.__name__
    return melhor_caminho, melhor_custo, heuristica_melhor_solucao


### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [165]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[0] = vizinho_mais_proximo(matriz_problema_km, 1)[0]
resultados_vmp_busca_local[0] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1]
Custo: 2291.3999999999996
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 48, 40, 36, 31, 7, 19, 34, 16, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 17, 25, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 44, 28, 15, 18, 45, 1]
Melhor custo pos busca local: 2222.5999999999995
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 48, 40, 36, 25, 31, 7, 19, 34, 16, 33, 39, 35, 42, 26, 32, 17, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 41, 30, 6, 27, 28, 15, 18, 45, 44, 1]
Melhor custo pos busca local: 2128.2
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1


### Problema 2
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pelo tempo em minutos.


In [75]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[1] = vizinho_mais_proximo(matriz_problema_min, 1)[0]
resultados_vmp_busca_local[1] = melhor_caminho


Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1]
Custo: 2355.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 19, 34, 16, 32, 26, 42, 35, 33, 20, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 27, 6, 30, 41, 47, 39, 1]
Melhor custo pos busca local: 2291.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 48, 40, 25, 17, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 7, 31, 36, 22, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 27, 6, 30, 41, 5, 47, 38, 1]
Melhor custo pos busca local: 2230.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 23, 41, 30, 6, 2

### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância
em km

In [76]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, vizinho_mais_proximo, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[2] = vizinho_mais_proximo(matriz_problema_km_36, 1)[0]
resultados_vmp_busca_local[2] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1]
Custo: 1951.4999999999998
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 36, 25, 31, 7, 34, 16, 33, 20, 17, 19, 32, 26, 35, 13, 24, 29, 14, 5, 18, 15, 28, 27, 6, 30, 1]
Melhor custo pos busca local: 1781.9999999999998
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 36, 25, 31, 17, 7, 34, 16, 33, 20, 19, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 8, 1]
Melhor custo pos busca local: 1772.8999999999996
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 21, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 8

### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto
em minutos.


In [77]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[3] = vizinho_mais_proximo(matriz_problema_min_36, 1)[0]
resultados_vmp_busca_local[3] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1]
Custo: 1950.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 23, 4, 3, 22, 36, 25, 31, 7, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 21, 17, 20, 1]
Melhor custo pos busca local: 1876.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 23, 4, 3, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 21, 25, 17, 20, 1]
Melhor custo pos busca local: 1859.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 30, 6, 27, 28, 15, 18, 14, 29, 24, 13, 35, 33, 16, 34, 26, 32, 19, 7, 31, 36, 22, 25, 17, 20, 1]
Melhor custo pos busca local: 1

### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km


In [78]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[4] = vizinho_mais_proximo(matriz_problema_km_24, 1)[0]
resultados_vmp_busca_local[4] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Custo: 1379.3
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1352.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Melhor custo pos busca local: 1337.3000000000004
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Melhor custo pos busca local: 1328.6000000000001

Melhor caminho: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Melhor custo: 1328.6000000000001
Heuristica: inversao


### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos



In [79]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[5] = vizinho_mais_proximo(matriz_problema_min_24, 1)[0]
resultados_vmp_busca_local[5] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1]
Custo: 1283.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 1]
Melhor custo pos busca local: 1255.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 2, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Melhor custo pos busca local: 1242.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Melhor custo pos busca local: 1235.0

Melhor caminho: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Melhor custo: 1235.0
Heuristica: inversao


### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [80]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[6] = vizinho_mais_proximo(matriz_problema_km_12, 1)[0]
resultados_vmp_busca_local[6] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 10, 12, 11, 2, 3, 4, 5, 7, 6, 1]
Custo: 828.4
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 2, 3, 4, 6, 7, 5, 1]
Melhor custo pos busca local: 772.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 10, 11, 2, 3, 4, 6, 12, 7, 5, 1]
Melhor custo pos busca local: 745.7
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 11, 10, 8, 9, 12, 7, 5, 1]
Melhor custo pos busca local: 707.9000000000001

Melhor caminho: [1, 6, 4, 3, 2, 11, 10, 8, 9, 12, 7, 5, 1]
Melhor custo: 707.9000000000001
Heuristica: inversao


### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [81]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[7] = vizinho_mais_proximo(matriz_problema_min_12, 1)[0]
resultados_vmp_busca_local[7] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 6, 7, 1]
Custo: 687.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 8, 1]
Melhor custo pos busca local: 649.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Melhor custo pos busca local: 609.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Melhor custo pos busca local: 609.0

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 6, 4, 3, 1]
Melhor custo: 609.0
Heuristica: shift


### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km


In [154]:
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

sub_matriz = matriz_problema_km[cidades_7].iloc[indices_7].copy()
mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

matriz_normalizada = sub_matriz.reset_index(drop=True)
matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
    matriz_normalizada,
    1,
    vizinho_mais_proximo,
    [swap, shift, inversao]
)

melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

print(f"\nMelhor caminho: {melhor_caminho_real}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heurística: {heuristica_melhor_solucao}")

resultados_vmp[8] = vizinho_mais_proximo(matriz_normalizada, 1)[0]
resultados_vmp_busca_local[8] = melhor_caminho_real

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 3, 4, 5, 7, 6, 2, 1]
Custo: 518.5
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo pos busca local: 438.3
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo pos busca local: 438.3
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo pos busca local: 438.3

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 1]
Melhor custo: 438.3
Heurística: swap


### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [149]:
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

sub_matriz = matriz_problema_min[cidades_7].iloc[indices_7].copy()

mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

matriz_normalizada = sub_matriz.reset_index(drop=True)
matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
    matriz_normalizada,
    1,
    vizinho_mais_proximo,
    [swap, shift, inversao]
)

melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

print(f"\nMelhor caminho: {melhor_caminho_real}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[9] = vizinho_mais_proximo(matriz_normalizada, 1)[0]
resultados_vmp_busca_local[9] = melhor_caminho_real

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 3, 4, 7, 5, 6, 2, 1]
Custo: 413.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo pos busca local: 364.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo pos busca local: 364.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 2, 7, 5, 6, 1]
Melhor custo pos busca local: 364.0

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 1]
Melhor custo: 364.0
Heuristica: swap


### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [150]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1,7))].iloc[0:6].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[10] = vizinho_mais_proximo(matriz_problema_km_6, 1)[0]
resultados_vmp_busca_local[10] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 3, 4, 2, 5, 6, 1]
Custo: 463.6
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 348.3
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 348.3
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 348.3

Melhor caminho: [1, 5, 2, 3, 4, 6, 1]
Melhor custo: 348.3
Heuristica: swap


### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [151]:

# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1,7))].iloc[0:6].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6, 1, vizinho_mais_proximo, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_vmp[11] = vizinho_mais_proximo(matriz_problema_min_6, 1)[0]
resultados_vmp_busca_local[11] = melhor_caminho

Aplicando Heuristica Construtiva:  vizinho_mais_proximo
Caminho: [1, 3, 4, 2, 5, 6, 1]
Custo: 400.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 305.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 305.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 305.0

Melhor caminho: [1, 5, 2, 3, 4, 6, 1]
Melhor custo: 305.0
Heuristica: swap


In [153]:
print(resultados_vmp)
print(resultados_vmp_busca_local)

[[1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 48, 40, 25, 17, 7, 19, 16, 34, 33, 20, 38, 37, 47, 39, 35, 42, 26, 32, 31, 36, 13, 24, 29, 43, 14, 46, 5, 41, 6, 27, 30, 44, 28, 18, 15, 45, 1], [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 40, 48, 25, 17, 20, 19, 16, 32, 26, 42, 39, 33, 34, 7, 31, 36, 38, 37, 13, 24, 29, 43, 14, 46, 45, 18, 15, 28, 44, 6, 30, 27, 41, 47, 35, 1], [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 25, 17, 7, 19, 16, 34, 33, 20, 31, 36, 32, 26, 35, 13, 24, 29, 14, 5, 28, 18, 15, 6, 27, 30, 1], [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 25, 17, 20, 19, 16, 32, 26, 34, 33, 35, 13, 24, 29, 14, 18, 15, 28, 6, 30, 27, 7, 31, 36, 1], [1, 8, 9, 10, 12, 11, 2, 3, 4, 23, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 1], [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 23, 21, 22, 17, 20, 19, 16, 7, 13, 24, 14, 18, 15, 6, 1], [1, 8, 9, 10, 12, 11, 2, 3, 4, 5, 7, 6, 1], [1, 8, 9, 12, 10, 11, 5, 2, 3, 4, 6, 7, 1], ([1, 3, 4, 5, 7, 6, 2, 1], np.float64(518.5)), [1, 3, 4, 7, 5, 6, 2, 1]

### Insercao Mais Barata

In [135]:
import math

# Estamos considerando o label numerico
def insercao_mais_barata(grafo : pd.DataFrame, vertice_inicial : int | str):

    vertices_restantes = set(grafo.index)

    # Selecionamos o indice do vertice mais proximo a raiz de forma gulosa
    v_mais_proximo = grafo[vertice_inicial].idxmin()

    # Ciclo inicial
    rota = [vertice_inicial, v_mais_proximo + 1, vertice_inicial]
    vertices_restantes.remove( vertice_inicial - 1, )
    vertices_restantes.remove(v_mais_proximo)
    vertices_inseridos = {vertice_inicial - 1, v_mais_proximo}

    custo_total = 0
    while vertices_restantes:
        # Estamos selecionando o vertice de 'vertices_restantes' que tem a menor distancia a outro vertice
        # Filtra apenas as linhas da matriz referentes aos vertices do conjunto de vertices_restantes
        r = grafo[list(vertices_restantes)].iloc[list(vertices_inseridos)].min().idxmin()

        # Consegue o indice do vertice do conjunto 'vertices_restantes' que tem a menor distancia
        # ao algum vertice incluido na rota. Para manter este codigo similar ao pseudocodigo visto em sala, chamaremos a variavel de
        # Encontrar a melhor posicao para inserir a rota
        melhor_custo_insercao = math.inf
        melhor_posicao_insercao = None
        for i in range(1, len(rota) - 1):
            u, v = rota[i] - 1, rota[i + 1] - 1
            custo = grafo[u + 1][r] + grafo[r + 1][v] - grafo[u + 1][v]
            if custo < melhor_custo_insercao:
                melhor_custo_insercao = custo
                melhor_posicao_insercao = i

        rota.insert(melhor_posicao_insercao + 1, r + 1)
        vertices_restantes.remove(r)
        vertices_inseridos.add(r)
        custo_total += melhor_custo_insercao
    return rota, calcular_custo(rota, grafo)



In [152]:
resultados_imb = [None] * 12
resultados_imb_busca_local = [None] * 12

### Problema 1
Percurso por 48 cidades, partindo de ANGICOS, com funcao custo definida pela distancia em
km.


In [168]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[0] = insercao_mais_barata(matriz_problema_km, 1)[0]
resultados_imb_busca_local[0] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Custo: 2148.1
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 21, 9, 11, 10, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1]
Melhor custo pos busca local: 2115.5999999999995
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 21, 9, 10, 12, 22, 48, 40, 36, 25, 31, 17, 7, 19, 34, 16, 32, 26, 42, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 11, 2, 23, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Melhor custo pos busca local: 2078.2999999999997
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1

### Problema 2

Percurso por 48 cidades, partindo de ANGICOS, com função custo definida pelo tempo

In [169]:
melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min, 1, insercao_mais_barata, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[1] = insercao_mais_barata(matriz_problema_min, 1)[0]
resultados_imb_busca_local[1] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 33, 39, 35, 42, 32, 26, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 3, 4, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Custo: 2197.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 39, 33, 35, 42, 26, 32, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 4, 3, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Melhor custo pos busca local: 2167.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 39, 35, 42, 26, 32, 16, 34, 33, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 3, 4, 2, 11, 5, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1]
Melhor custo pos busca local: 2119.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 38, 20, 

### Problema 3
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a distância
em km

In [170]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_km_36 = matriz_problema_km[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_36, 1, insercao_mais_barata, [swap, shift, inversao])
print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[2] = insercao_mais_barata(matriz_problema_km_36, 1)[0]
resultados_imb_busca_local[2] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 21, 9, 12, 10, 11, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Custo: 1835.3
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 21, 9, 11, 10, 12, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Melhor custo pos busca local: 1814.8999999999999
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 21, 9, 10, 12, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 11, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1]
Melhor custo pos busca local: 1786.6000000000001
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 10, 12, 9, 8, 21, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 4, 3, 2, 11, 5, 23, 1]
Melhor 

### Problema 4
Percurso por 36 cidades, partindo de ANGICOS, tomando como função custo a o tempo de trajeto
em minutos.


In [171]:
# Filtra as colunas do DataFrame para obter o intervalo de cidades
# [1,36]
matriz_problema_min_36 = matriz_problema_min[list(range(1,37))].iloc[0:36].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_36, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"Melhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[3] = insercao_mais_barata(matriz_problema_min_36, 1)[0]
resultados_imb_busca_local[3] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Custo: 1931.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Melhor custo pos busca local: 1931.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 11, 5, 14, 29, 24, 13, 20, 17, 25, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Melhor custo pos busca local: 1828.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 10, 20, 17, 25, 22, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 13, 24, 29, 14, 5, 11, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1]
Melhor custo pos busca local: 1

### Problema 5
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo a distância em km


In [172]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_km_24 = matriz_problema_km[list(range(1,25))].iloc[0:24].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_24, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[4] = insercao_mais_barata(matriz_problema_km_24, 1)[0]
resultados_imb_busca_local[4] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 9, 12, 10, 11, 21, 22, 17, 7, 16, 19, 20, 13, 24, 14, 5, 2, 3, 4, 6, 15, 18, 23, 1]
Custo: 1459.3000000000002
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 2, 3, 4, 18, 15, 6, 23, 1]
Melhor custo pos busca local: 1405.1000000000001
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Melhor custo pos busca local: 1390.5000000000002
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Melhor custo pos busca local: 1390.5000000000002

Melhor caminho: [1, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 5, 11, 2, 3, 4, 18, 15, 6, 23, 1]
Melhor custo: 1390.50000000000

### Problema 6
Percurso por 24 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos



In [173]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 24]
matriz_problema_min_24 = matriz_problema_min[list(range(1,25))].iloc[0:24].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_24, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[5] = insercao_mais_barata(matriz_problema_min_24, 1)[0]
resultados_imb_busca_local[5] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 9, 12, 24, 13, 20, 19, 16, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1]
Custo: 1355.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1]
Melhor custo pos busca local: 1351.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Melhor custo pos busca local: 1260.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Melhor custo pos busca local: 1260.0

Melhor caminho: [1, 8, 9, 12, 24, 13, 20, 16, 19, 7, 17, 22, 21, 23, 2, 3, 4, 6, 15, 18, 14, 5, 11, 10, 1]
Melhor custo: 1260.0
Heuristica: shift


### Problema 7
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo a distância em km

In [174]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_km_12 = matriz_problema_km[list(range(1,13))].iloc[0:12].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_12, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[6] = insercao_mais_barata(matriz_problema_km_12, 1)[0]
resultados_imb_busca_local[6] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo: 711.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 711.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 711.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 11, 10, 12, 7, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 699.2

Melhor caminho: [1, 8, 9, 11, 10, 12, 7, 5, 2, 3, 4, 6, 1]
Melhor custo: 699.2
Heuristica: inversao


### Problema 8
Percurso por 12 cidades, partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [175]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 12]
matriz_problema_min_12 = matriz_problema_min[list(range(1,13))].iloc[0:12].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_12, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[7] = insercao_mais_barata(matriz_problema_min_12, 1)[0]
resultados_imb_busca_local[7] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Custo: 609.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 609.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 609.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo pos busca local: 609.0

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1]
Melhor custo: 609.0
Heuristica: None


### Problema 9
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em km


In [182]:
cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

sub_matriz = matriz_problema_km[cidades_7].iloc[indices_7].copy()
mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

matriz_normalizada = sub_matriz.reset_index(drop=True)
matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
    matriz_normalizada,
    1,
    insercao_mais_barata,
    [swap, shift, inversao]
)

melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

print(f"\nMelhor caminho: {melhor_caminho_real}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heurística: {heuristica_melhor_solucao}")

resultados_imb[8] = [mapa_traducao[cidade_ficticia] for cidade_ficticia in insercao_mais_barata(matriz_normalizada, 1)[0]]
resultados_imb_busca_local[8] = melhor_caminho_real

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Custo: 438.3
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Melhor custo pos busca local: 438.3
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Melhor custo pos busca local: 438.3
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Melhor custo pos busca local: 438.3

Melhor caminho: [1, 8, 9, 7, 12, 10, 11, 1]
Melhor custo: 438.3
Heurística: None


### Problema 10
Percurso por 7 cidades (1, 7, 8, 9, 10, 11, 12), com função custo em minutos


In [183]:
import pandas as pd

cidades_7 = [1, 7, 8, 9, 10, 11, 12]
indices_7 = [c - 1 for c in cidades_7]

sub_matriz = matriz_problema_min[cidades_7].iloc[indices_7].copy()

mapa_traducao = { i + 1: id_real for i, id_real in enumerate(cidades_7) }

matriz_normalizada = sub_matriz.reset_index(drop=True)
matriz_normalizada.columns = range(1, len(matriz_normalizada) + 1)

melhor_caminho_ficticio, melhor_custo, heuristica_melhor_solucao = aplicar_problema(
    matriz_normalizada,
    1,
    insercao_mais_barata,
    [swap, shift, inversao]
)

melhor_caminho_real = [mapa_traducao[cidade_ficticia] for cidade_ficticia in melhor_caminho_ficticio]

print(f"\nMelhor caminho (Original): {melhor_caminho_real}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heurística vencedora: {heuristica_melhor_solucao}")

resultados_imb[9] = [mapa_traducao[cidade_ficticia] for cidade_ficticia in insercao_mais_barata(matriz_normalizada, 1)[0]]
resultados_imb_busca_local[9] = melhor_caminho_real

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Custo: 364.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Melhor custo pos busca local: 364.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Melhor custo pos busca local: 364.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, np.int64(4), np.int64(2), np.int64(7), np.int64(5), np.int64(6), 1]
Melhor custo pos busca local: 364.0

Melhor caminho (Original): [1, 8, 9, 7, 12, 10, 11, 1]
Melhor custo: 364.0
Heurística vencedora: None


### Problema 11
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo a distância em km


In [179]:
# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_km_6 = matriz_problema_km[list(range(1,7))].iloc[0:6].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_km_6, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[10] = insercao_mais_barata(matriz_problema_km_6, 1)[0]
resultados_imb_busca_local[10] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 3, 4, 6, 2, 5, 1]
Custo: 348.5
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 5, 1]
Melhor custo pos busca local: 348.3
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 5, 1]
Melhor custo pos busca local: 348.3
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 6, 4, 3, 2, 5, 1]
Melhor custo pos busca local: 348.3

Melhor caminho: [1, 6, 4, 3, 2, 5, 1]
Melhor custo: 348.3
Heuristica: swap


### Problema 12
Percurso por 6 cidades (1 a 6), partindo de ANGICOS, tomando como função custo o tempo de trajeto em minutos

In [180]:

# Filtra as colunas e linhas do DataFrame para as cidades [1, 6]
matriz_problema_min_6 = matriz_problema_min[list(range(1,7))].iloc[0:6].copy()

melhor_caminho, melhor_custo, heuristica_melhor_solucao = aplicar_problema(matriz_problema_min_6, 1, insercao_mais_barata, [swap, shift, inversao])

print(f"\nMelhor caminho: {melhor_caminho}")
print(f"Melhor custo: {melhor_custo}")
print(f"Heuristica: {heuristica_melhor_solucao}")

resultados_imb[11] = insercao_mais_barata(matriz_problema_min_6, 1)[0]
resultados_imb_busca_local[11] = melhor_caminho

Aplicando Heuristica Construtiva:  insercao_mais_barata
Caminho: [1, 3, 4, 6, 2, 5, 1]
Custo: 305.0
Aplicando busca(s) local(is):
Heuristica : swap
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 6, 2, 5, 1]
Melhor custo pos busca local: 305.0
Heuristica : shift
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 6, 2, 5, 1]
Melhor custo pos busca local: 305.0
Heuristica : inversao
Tipo : Primeira Melhoria
Melhor caminho pos busca local: [1, 3, 4, 6, 2, 5, 1]
Melhor custo pos busca local: 305.0

Melhor caminho: [1, 3, 4, 6, 2, 5, 1]
Melhor custo: 305.0
Heuristica: None


In [184]:
print(resultados_imb)
print(resultados_imb_busca_local)

[[1, 8, 21, 9, 10, 11, 12, 22, 36, 48, 40, 25, 31, 17, 7, 19, 34, 16, 42, 26, 32, 35, 39, 33, 20, 38, 37, 47, 13, 24, 29, 43, 14, 46, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 45, 44, 41, 23, 1], [1, 8, 9, 12, 43, 29, 24, 13, 37, 47, 33, 39, 35, 42, 32, 26, 16, 34, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 10, 11, 5, 2, 3, 4, 14, 46, 45, 18, 15, 28, 44, 41, 27, 6, 30, 21, 23, 1], [1, 8, 21, 9, 12, 10, 11, 22, 36, 25, 31, 7, 19, 17, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 5, 2, 3, 4, 30, 6, 27, 28, 15, 18, 23, 1], [1, 8, 9, 12, 29, 24, 13, 20, 17, 25, 36, 31, 7, 19, 32, 26, 34, 16, 33, 35, 22, 10, 11, 5, 14, 2, 3, 4, 18, 15, 28, 27, 6, 30, 21, 23, 1], [1, 8, 9, 12, 10, 11, 21, 22, 17, 7, 16, 19, 20, 13, 24, 14, 5, 2, 3, 4, 6, 15, 18, 23, 1], [1, 8, 9, 12, 24, 13, 20, 19, 16, 7, 17, 22, 21, 23, 10, 11, 5, 14, 2, 3, 4, 6, 15, 18, 1], [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1], [1, 8, 9, 7, 12, 10, 11, 5, 2, 3, 4, 6, 1], [1, 8, 9, 7, 12, 10, 11, 1], [1, 8, 9, 7, 12, 10, 11, 1], [1, 3, 4, 6, 

## Algoritmo Genetico

### Gerador de solucoes aleatorias

In [19]:
import random
def solucao_aleatoria(grafo : pd.DataFrame, vertice_inicial):
    rota = list(map(lambda i : i + 1,grafo.index))
    rota.remove(vertice_inicial)
    random.shuffle(rota)
    return [vertice_inicial] + rota + [vertice_inicial]

# Funca para obter o custo total a partir de uma solucao (caminho)
def calcular_custo(solucao, grafo):
    if len(solucao) <= 1:
        return
    custo = 0
    for prox in range(1, len(solucao)):
        v1 = solucao[prox - 1]
        v2 = solucao[prox]
        # print("v1: ", v1)
        # print("v2: ", v2)
        custo += grafo[v1][v2 - 1]
    return custo

samples = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,6)]
print(samples)
# custos = [calcular_custo(solucao, matriz_problema_km) for solucao in samples]
# print(f"Minimo : {min(custos)}")
# print(f"Maximo : {max(custos)}")
#

[[1, 12, 26, 46, 44, 27, 2, 4, 36, 21, 8, 28, 9, 25, 41, 18, 24, 3, 32, 43, 10, 37, 15, 31, 22, 14, 16, 47, 35, 48, 20, 45, 13, 34, 23, 11, 5, 38, 40, 30, 7, 42, 33, 19, 29, 39, 17, 6, 1], [1, 2, 41, 15, 32, 34, 3, 40, 43, 29, 31, 33, 39, 23, 42, 5, 14, 26, 45, 12, 35, 30, 8, 13, 48, 24, 44, 37, 4, 19, 6, 7, 17, 9, 21, 25, 11, 10, 38, 16, 20, 28, 36, 47, 22, 18, 46, 27, 1], [1, 29, 28, 22, 37, 21, 6, 39, 18, 2, 36, 10, 26, 33, 3, 45, 34, 24, 19, 48, 4, 40, 20, 32, 16, 13, 42, 17, 11, 12, 30, 47, 23, 44, 7, 27, 15, 38, 8, 35, 5, 25, 43, 31, 46, 14, 41, 9, 1], [1, 31, 24, 14, 46, 42, 6, 7, 40, 34, 22, 28, 44, 36, 5, 39, 25, 37, 29, 43, 48, 32, 19, 16, 45, 38, 47, 23, 21, 30, 3, 15, 9, 35, 33, 4, 11, 17, 18, 8, 26, 12, 2, 27, 13, 20, 41, 10, 1], [1, 25, 19, 17, 40, 48, 13, 43, 9, 11, 47, 29, 32, 36, 33, 34, 31, 37, 12, 24, 3, 7, 38, 44, 23, 10, 41, 35, 27, 42, 22, 39, 2, 16, 20, 5, 6, 4, 14, 26, 21, 30, 45, 15, 18, 46, 28, 8, 1], [1, 5, 37, 6, 7, 32, 40, 3, 20, 25, 9, 43, 44, 8, 48, 18, 4

## Gerando populacao
Gerar 15 indivíduos

### Populacao para o problema 1

In [193]:
solucao_insercao_mais_barata = resultados_imb[0]
solucao_vizinho_mais_proximo = resultados_vmp[0]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[0]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[0]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km, 1) for i in range(0,36)]

pop_problema_1 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_1 = sorted(pop_problema_1, key=lambda solucao: calcular_custo(solucao, matriz_problema_km))
print(f"População problema 1 criada. Tamanho: {len(pop_problema_1)}")


População problema 1 criada. Tamanho: 40


### População para o problema 2

In [194]:
solucao_insercao_mais_barata = resultados_imb[1]
solucao_vizinho_mais_proximo = resultados_vmp[1]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[1]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[1]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_min, 1) for i in range(0,36)]

pop_problema_2 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_2 = sorted(pop_problema_2, key=lambda solucao: calcular_custo(solucao, matriz_problema_min))
print(f"População problema 2 criada. Tamanho: {len(pop_problema_2)}")

População problema 2 criada. Tamanho: 40


### População para o problema 3

In [195]:
solucao_insercao_mais_barata = resultados_imb[2]
solucao_vizinho_mais_proximo = resultados_vmp[2]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[2]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[2]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km_36, 1) for i in range(0,36)]

pop_problema_3 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_3 = sorted(pop_problema_3, key=lambda solucao: calcular_custo(solucao, matriz_problema_km_36))
print(f"População problema 3 criada. Tamanho: {len(pop_problema_3)}")

População problema 3 criada. Tamanho: 40


### População para o problema 4

In [196]:
solucao_insercao_mais_barata = resultados_imb[3]
solucao_vizinho_mais_proximo = resultados_vmp[3]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[3]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[3]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_min_36, 1) for i in range(0,36)]

pop_problema_4 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_4 = sorted(pop_problema_4, key=lambda solucao: calcular_custo(solucao, matriz_problema_min_36))
print(f"População problema 4 criada. Tamanho: {len(pop_problema_4)}")

População problema 4 criada. Tamanho: 40


### População para o problema 5

In [198]:
solucao_insercao_mais_barata = resultados_imb[4]
solucao_vizinho_mais_proximo = resultados_vmp[4]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[4]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[4]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km_24, 1) for i in range(0,36)]

pop_problema_5 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_5 = sorted(pop_problema_5, key=lambda solucao: calcular_custo(solucao, matriz_problema_km_24))
print(f"População problema 5 criada. Tamanho: {len(pop_problema_5)}")

População problema 5 criada. Tamanho: 40


### População para o problema 6

In [199]:
solucao_insercao_mais_barata = resultados_imb[5]
solucao_vizinho_mais_proximo = resultados_vmp[5]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[5]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[5]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_min_24, 1) for i in range(0,36)]

pop_problema_6 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_6 = sorted(pop_problema_6, key=lambda solucao: calcular_custo(solucao, matriz_problema_min_24))
print(f"População problema 6 criada. Tamanho: {len(pop_problema_6)}")

População problema 6 criada. Tamanho: 40


### População para o problema 7

In [200]:
solucao_insercao_mais_barata = resultados_imb[6]
solucao_vizinho_mais_proximo = resultados_vmp[6]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[6]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[6]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km_12, 1) for i in range(0,36)]

pop_problema_7 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_7 = sorted(pop_problema_7, key=lambda solucao: calcular_custo(solucao, matriz_problema_km_12))
print(f"População problema 7 criada. Tamanho: {len(pop_problema_7)}")

População problema 7 criada. Tamanho: 40


### População para o problema 8

In [201]:
solucao_insercao_mais_barata = resultados_imb[7]
solucao_vizinho_mais_proximo = resultados_vmp[7]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[7]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[7]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_min_12, 1) for i in range(0,36)]

pop_problema_8 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                        solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_8 = sorted(pop_problema_8, key=lambda solucao: calcular_custo(solucao, matriz_problema_min_12))
print(f"População problema 8 criada. Tamanho: {len(pop_problema_8)}")

População problema 8 criada. Tamanho: 40


### População para o problema 9

### População para o problema 10

### População para o problema 11

In [202]:
solucao_insercao_mais_barata = resultados_imb[10]
solucao_vizinho_mais_proximo = resultados_vmp[10]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[10]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[10]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_km_6, 1) for i in range(0,36)]

pop_problema_11 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                         solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_11 = sorted(pop_problema_11, key=lambda solucao: calcular_custo(solucao, matriz_problema_km_6))
print(f"População problema 11 criada. Tamanho: {len(pop_problema_11)}")

População problema 11 criada. Tamanho: 40


### População para o problema 12

In [203]:
solucao_insercao_mais_barata = resultados_imb[11]
solucao_vizinho_mais_proximo = resultados_vmp[11]
solucao_insercao_mais_barata_local = resultados_imb_busca_local[11]
solucao_vizinho_mais_proximo_local = resultados_vmp_busca_local[11]

solucoes_aleatorias = [solucao_aleatoria(matriz_problema_min_6, 1) for i in range(0,36)]

pop_problema_12 = solucoes_aleatorias + [solucao_insercao_mais_barata, solucao_vizinho_mais_proximo, 
                                         solucao_insercao_mais_barata_local, solucao_vizinho_mais_proximo_local]

pop_problema_12 = sorted(pop_problema_12, key=lambda solucao: calcular_custo(solucao, matriz_problema_min_6))
print(f"População problema 12 criada. Tamanho: {len(pop_problema_12)}")

População problema 12 criada. Tamanho: 40


## 1) Inicio
Organizando os cromossomos

In [212]:
cromossomos_problema1 = [(solucao, calcular_custo(solucao, matriz_problema_km)) for solucao in pop_problema_1]
cromossomos_problema2 = [(solucao, calcular_custo(solucao, matriz_problema_min)) for solucao in pop_problema_2]
cromossomos_problema3 = [(solucao, calcular_custo(solucao, matriz_problema_km_36)) for solucao in pop_problema_3]
cromossomos_problema4 = [(solucao, calcular_custo(solucao, matriz_problema_min_36)) for solucao in pop_problema_4]
cromossomos_problema5 = [(solucao, calcular_custo(solucao, matriz_problema_km_24)) for solucao in pop_problema_5]
cromossomos_problema6 = [(solucao, calcular_custo(solucao, matriz_problema_min_24)) for solucao in pop_problema_6]
cromossomos_problema7 = [(solucao, calcular_custo(solucao, matriz_problema_km_12)) for solucao in pop_problema_7]
cromossomos_problema8 = [(solucao, calcular_custo(solucao, matriz_problema_min_12)) for solucao in pop_problema_8]
cromossomos_problema11 = [(solucao, calcular_custo(solucao, matriz_problema_km_6)) for solucao in pop_problema_11]
cromossomos_problema12 = [(solucao, calcular_custo(solucao, matriz_problema_min_6)) for solucao in pop_problema_12]

## Metodos de Selecao

In [205]:
# def elitismo(populacao : list, num_pares):
#     populacao_ordenada = sorted(populacao, key=lambda x: x[1])
#     genitores = []
#     for i in range(0,num_pares):
#         genitores.append((populacao_ordenada[i], populacao_ordenada[i + 1]))
#     return genitores

def elitismo(populacao : list, num_pares):
    populacao_ordenada = sorted(populacao, key=lambda x: x[1])
    genitores = []
    for i in range(0,num_pares):
        segundo_genitor = random.choice(range(0, num_pares))
        if segundo_genitor == i:
            segundo_genitor = 0
        genitores.append((populacao_ordenada[i], populacao_ordenada[segundo_genitor]))
    return genitores

def torneio(populacao: list, num_pares: int, tamanho_torneio: int = 3):
  genitores = []
  for _ in range(num_pares):
    competidores_1 = random.sample(populacao, k=tamanho_torneio)
    pai1 = min(competidores_1, key=lambda x: x[1])
    competidores_2 = random.sample(populacao, k=tamanho_torneio)
    pai2 = min(competidores_2, key=lambda x: x[1])
    genitores.append((pai1, pai2))
  return genitores


### Metodos de Cruzamento

In [206]:
def crossover_1_ponto(cromossomo_1, cromossomo_2):
    n = len(cromossomo_1) # Cromossomos tem comprimentos iguais
    # Ponto de corte
    mid = n // 2
    offspring_1 = cromossomo_1[0:mid] + cromossomo_2[mid:n]
    offspring_2 = cromossomo_2[0:mid] + cromossomo_1[mid:n]
    offsprings = [offspring_1, offspring_2]
    # As labels sao numeros em sequencia
    vertices = set(range(2, n))
    for i, offspring in enumerate(offsprings):
        # vertices ja presentes no offspring atual
        vertices_no_offspring = set(offspring)
        vertices_no_offspring.remove(1)
        # se o numero de elementos unicos eh menor que o tamanho - 1 (ja que eh um ciclo),
        # ja elementos repetidos
        if len(vertices_no_offspring) < n-1:
            vertices_restantes = vertices.difference(vertices_no_offspring)
            novo_offspring = offspring[0:mid]
            controle = set(offspring[1:mid])
            for v in offspring[mid:]:
                novo_v = None
                if v in controle:
                    novo_v = vertices_no_offspring.pop()
                else:
                    novo_v = v
                controle.add(novo_v)
                novo_offspring.append(novo_v)
            offsprings[i] = novo_offspring
    return offsprings[0], offsprings[1]

## Order Crossover (ANTIGO)
# def crossover_ox(genitor_1, genitor_2):
#     offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2

## Order Crossover (NOVO)
def crossover_ox(genitor_1, genitor_2):
    ponto = random.choice(range(1, len(genitor_1) - 1))
    offspring_1, offspring_2 = genitor_1[:ponto], genitor_2[:ponto]

    vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
    for i in genitor_2:
        if i not in vertices_incluidos1:
            offspring_1.append(i)
    offspring_1.append(genitor_1[0])
    for i in genitor_1:
        if i not in vertices_incluidos2:
            offspring_2.append(i)
    offspring_2.append(genitor_2[0])
    return offspring_1, offspring_2

## Partially-Bmapped Crossover (à finalizar)
# https://www.nature.com/articles/s41598-025-86695-4
# def crossover_pbx(genitor_1, genitor_2):
#     posicoes = random.randint(1, len(genitor_1) - 1)
#     pos_genes_1 = set(random.choices(range(1, len(genitor_1) - 1), k=posicoes))
#     pos_genes_2 = set(range(0,48)).difference(genes_1)
#     genes_1 = {genitor_1[i] for i in pos_genes_1}
#     genes_2 = {}
#     offspring_1, offspring_2 = [0 for i in range(48)], [0 for i in range(48)]
#     for i in genes:
#         offspring_1[i] = genitor_1[i]
#     for i in range(0,48):
#         if i in pos_genes_1:
#             continue
#         if genitor_2[i] in genes_1:
#             g = genes_2.pop()
#             offspring_1[i] = g 
#     genes_1 = random.choices(range(1, len(genitor_1) - 1), k=posicoes)
#     offspring_1, offspring_2 = genitor_1[:ponto], genitor_2[:ponto]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2

# def crossover_2_pontos(genitor_1, genitor_2):
#     offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     g1, g2 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_1_inicio, g_1_final = min(g1,g2), max(g1,g2)
#     g3, g4 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_2_inicio, g_2_final = min(g1,g2), max(g1,g2)
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2
#     pass

# def crossover_2_pontos(genitor_1, genitor_2):
#     offspring_1, offspring_2 = genitor_1[:len(genitor_1) // 2], genitor_2[:len(genitor_2) // 2]
#     vertices_incluidos1, vertices_incluidos2 = set(offspring_1), set(offspring_2)
#     g1, g2 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_1_inicio, g_1_final = min(g1,g2), max(g1,g2)
#     g3, g4 = random.sample(range(1,len(genitor_1) - 1), 2)
#     g_2_inicio, g_2_final = min(g1,g2), max(g1,g2)
#     segmento_1 = genitor_1[g1:g2]
#     vertices_segmento_1 = set(segmento_1)
#     vertices_segmento_2 = set(segmento_2)
#     for v in segmento_2:
#         if v in segmento_1:
#
#     segmento_2 = genitor_2[g3:g4]
#
#
#     for i in genitor_2:
#         if i not in vertices_incluidos1:
#             offspring_1.append(i)
#     offspring_1.append(genitor_1[0])
#     for i in genitor_1:
#         if i not in vertices_incluidos2:
#             offspring_2.append(i)
#     offspring_2.append(genitor_2[0])
#     return offspring_1, offspring_2
#     pass

def crossover_uniforme(genitor_1,genitor_2):
    offspring_1, offspring_2 = genitor_1, genitor_2
    genes_faltantes_1, genes_faltantes_2 = set(range(0, 49)), set(range(0, 49))
    for i in range(1, len(offspring_1) - 1):
        if random.random() < 0.5:
            if offspring_2[i] not in genes_faltantes_1:
                offspring_1[i] = genes_faltantes_1.pop()
            else:
                offspring_1[i] = offspring_2[i]
            genes_faltantes_1.add(offspring_1[i])
        if random.random() < 0.5:
            if offspring_1[i] not in genes_faltantes_2:
                offspring_2[i] = genes_faltantes_2.pop()
            else:
                offspring_2[i] = offspring_1[i]
            genes_faltantes_2.add(offspring_2[i])
    return offspring_1, offspring_2

def cruzamento(genitores, grafo):
    offsprings = []
    for g in genitores:
        genitor_1,genitor_2=g
        offspring1, offspring2 = crossover_ox(genitor_1[0], genitor_2[0])
        # print(f"Genitores: {genitor_1[0]} e {genitor_2[0]}")
        # print(f"Offsprings: {offspring1[0]} e {offspring2[0]}")
        offsprings.extend([(offspring1, calcular_custo(offspring1, grafo)), (offspring2,calcular_custo(offspring1, grafo))])
    return offsprings

In [207]:
import random
def mutacao_swap(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo in cromossomos:
        cromossomo = list(cromossomo[0]) # extrai o caminho

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            cromossomo[gene_1], cromossomo[gene_2] = cromossomo[gene_2], cromossomo[gene_1]
        # Adiciona o cromossomo (mutado ou não) com seu novo custo
        novos_cromossomos.append((cromossomo, calcular_custo(cromossomo, grafo)))

    return novos_cromossomos

def mutacao_inversao(cromossomos, probabilidade, grafo):
    novos_cromossomos = []
    for cromossomo_original_tuple in cromossomos:
        cromossomo = list(cromossomo_original_tuple[0]) # Trabalhar com a lista da rota

        if random.random() < probabilidade:
            gene_1, gene_2 = random.sample(range(1, len(cromossomo) - 1), 2)
            gene_inicio = min(gene_1, gene_2)
            gene_fim = max(gene_1, gene_2)
            parte_invertida = cromossomo[gene_inicio:gene_fim][::-1]
            novo = cromossomo[:gene_inicio] + parte_invertida + cromossomo[gene_fim:]

            novos_cromossomos.append((novo, calcular_custo(novo,grafo)))
            # print(f"Custo mutado : {calcular_custo(novo, grafo)}")
        else:
            novos_cromossomos.append(cromossomo_original_tuple)
        # Adiciona o cromossomo (mutado ou não) com seu novo custo

    return novos_cromossomos




In [ ]:
# for custo, cromossomo in cromossomos.items():
#     mutacao_swap(cromossomo, 0.5)

In [208]:
a = {1:"a"}
b = random.choice(list(a.items()))
b

(1, 'a')

In [209]:
# Renovacao da populacao por elitismo
def renovacao_elitismo(populacao_atual, offspring, num_populacao):
    pop = populacao_atual + offspring
    pop_ordenada = sorted(pop, key=lambda x: x[1])
    return pop_ordenada[:num_populacao]

# Renovacao da populacao por torneio
def renovacao_torneio(populacao_atual, offspring, num_populacao):
    # Usando torneio
    nova_populacao = []
    pop = populacao_atual + offspring
    while len(nova_populacao) < num_populacao:
        candidatos = random.choices(pop, k=3)
        # offspring_individuo = random.choice(list(offspring))
        escolhido = min(candidatos, key=lambda x: x[1])
        nova_populacao.append(escolhido)
    return nova_populacao

def roleta(populacao, offspring, num_populacao):
    populacao = populacao + offspring
    cromossomos = []
    somatorio_custo = sum(map(lambda cromossomo : cromossomo[1], populacao))
    values = [solucao[1]/somatorio_custo for solucao in populacao]
    fitness = []
    for val in values:
        min_val = min(values)
        max_val = max(values)
        normalized_data = (max_val - val) / (max_val - min_val)
        fitness.append(normalized_data)
    print(f"Custos : ", list(map(lambda x : x[1], populacao)))
    print(f"Fitness : ", fitness)
    solucoes = list(map(lambda c : c[1], populacao))
    return random.choices(populacao, weights=fitness, k=num_populacao)


In [210]:

print(float(random.random()))

0.9367712776852497


In [211]:
def algoritmo_genetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao, mutacao, selecao, cruzamento, renovacao):
    populacao_atual = populacao_inicial
    for iter in range(0, numero_iteracoes):
        genitores = selecao(populacao_atual, len(populacao_atual))
        # print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        # Ja aplica a taxa de mutacao a populacao
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)
        # print(f"Tamanho do offspring: {len(offsprings)}")
        # print(offsprings)
        populacao_atual = renovacao(populacao_atual, offsprings, len(populacao_atual))
        # print(f"Tamanho da pop: {len(populacao_atual)}")
    return populacao_atual

In [ ]:
import time
import json

def executar_problema_e_salvar(num_problema, matriz, populacao, nome_arquivo, taxa_mutacao=0.4, num_execucoes=20, num_geracoes=100):
    print(f"EXECUTANDO PROBLEMA {num_problema}...\n")
    
    melhores = [] # Lista para armazenar os melhores resultados de cada execução
    tempos = [] # Lista para armazenar os tempos de cada execução
    
    for i in range(num_execucoes):
        print(f"  Execução {i+1}/{num_execucoes}...", end=" ")
        tempo_comeco = time.perf_counter()

        resultado = algoritmo_genetico( # Executando o Algoritmo Genético
            matriz, 
            populacao, 
            num_geracoes, 
            0.4, 
            mutacao_inversao, 
            elitismo, 
            cruzamento, 
            renovacao_elitismo
        )
        tempo_fim = time.perf_counter()
        tempos.append(tempo_fim - tempo_comeco)
        
        melhor_execucao = min(resultado, key=lambda x: x[1]) # Melhor solução da execução
        melhores.append(melhor_execucao)
        
        print(f"Concluído em {(tempo_fim - tempo_comeco):.2f}s (Custo: {melhor_execucao[1]:.2f})")
    
    melhores_ordenados = sorted(melhores, key=lambda x: x[1]) # Ordenar por custo
    
    dados_salvar = {
        "numero_problema": num_problema,
        "num_execucoes": num_execucoes,
        "num_geracoes": num_geracoes,
        "taxa_mutacao": taxa_mutacao,
        "operador_mutacao": "mutacao_inversao",
        "operador_selecao": "elitismo",
        "operador_cruzamento": "cruzamento",
        "operador_renovacao": "renovacao_elitismo",
        "data_execucao": time.strftime("%Y-%m-%d %H:%M:%S"),
        "resultados": []
    }
    
    # Adicionar cada resultado
    for idx, (rota, custo) in enumerate(melhores_ordenados):
        dados_salvar["resultados"].append({
            "posicao": idx + 1,
            "custo": float(custo),
            "rota": [int(v) for v in rota],
            "tempo_execucao": float(tempos[idx]) if idx < len(tempos) else 0.0
        })
    
    # Calcular estatísticas
    custos = [custo for _, custo in melhores]
    dados_salvar["estatisticas"] = {
        "melhor_custo": float(min(custos)),
        "pior_custo": float(max(custos)),
        "custo_medio": float(sum(custos) / len(custos)),
        "tempo_medio": float(sum(tempos) / len(tempos)),
        "tempo_total": float(sum(tempos)),
        "desvio_padrao_custo": float((sum((c - sum(custos)/len(custos))**2 for c in custos) / len(custos))**0.5)
    }
    
    # Salvar em arquivo JSON
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        json.dump(dados_salvar, f, indent=2, ensure_ascii=False)
    
    # Salvar também em formato texto legível
    nome_txt = nome_arquivo.replace('.json', '.txt')
    with open(nome_txt, 'w', encoding='utf-8') as f:
        f.write(f"{'='*60}\n")
        f.write(f"RESULTADOS - PROBLEMA {num_problema}\n")
        f.write(f"{'='*60}\n\n")
        
        f.write(f"CONFIGURAÇÃO:\n")
        f.write(f"  Número de execuções: {num_execucoes}\n")
        f.write(f"  Gerações por execução: {num_geracoes}\n")
        f.write(f"  Taxa de mutação: 0.4\n")
        f.write(f"  Data da execução: {dados_salvar['data_execucao']}\n\n")
        
        f.write(f"{'='*60}\n")
        f.write(f"ESTATÍSTICAS GERAIS\n")
        f.write(f"{'='*60}\n")
        f.write(f"Melhor custo encontrado: {dados_salvar['estatisticas']['melhor_custo']:.2f}\n")
        f.write(f"Pior custo encontrado: {dados_salvar['estatisticas']['pior_custo']:.2f}\n")
        f.write(f"Custo médio: {dados_salvar['estatisticas']['custo_medio']:.2f}\n")
        f.write(f"Desvio padrão do custo: {dados_salvar['estatisticas']['desvio_padrao_custo']:.2f}\n")
        f.write(f"Tempo médio por execução: {dados_salvar['estatisticas']['tempo_medio']:.2f}s\n")
        f.write(f"Tempo total: {dados_salvar['estatisticas']['tempo_total']:.2f}s\n\n")
        
        f.write(f"{'='*60}\n")
        f.write(f"MELHORES SOLUÇÕES ENCONTRADAS\n")
        f.write(f"{'='*60}\n")
        
        for idx, (rota, custo) in enumerate(melhores_ordenados[:10]):  # Mostrar top 10
            f.write(f"\nTop {idx+1} {'-'*50}\n")
            f.write(f"Custo: {custo:.2f}\n")
            f.write(f"Rota: {rota}\n")
            if idx < len(tempos):
                f.write(f"Tempo: {tempos[idx]:.2f}s\n")
        
        if len(melhores_ordenados) > 10:
            f.write(f"\n[...] +{len(melhores_ordenados)-10} soluções adicionais\n")
    
    print(f"\nResultados salvos em:")
    print(f"  JSON: {nome_arquivo}")
    print(f"  TXT: {nome_txt}")
    
    return melhores_ordenados, dados_salvar["estatisticas"]

### Problema 1

In [218]:
## Problema 1
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_km, cromossomos_problema1, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_1_km.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 1\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_1_km.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 10, 12, 9, 8, 23, 4, 3, 2, 11, 5, 46, 14, 43, 29, 24, 13, 47, 37, 38, 20, 33, 39, 35, 42, 26, 32, 16, 34, 19, 7, 17, 31, 25, 36, 40, 48, 22, 21, 30, 6, 27, 28, 15, 18, 45, 44, 41, 1]
Custo: 2018.8999999999996
---------------------------

Custo médio:  2044.6399999999999
Tempo médio:  1.3005823699524626

Resultados salvos em: resultado_problema_1_km.txt


### Problema 2

In [219]:
## Problema 2
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_min, cromossomos_problema2, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_2_min.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 2\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_2_min.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 8, 9, 21, 30, 6, 27, 41, 44, 28, 15, 18, 45, 46, 14, 43, 29, 24, 13, 37, 47, 39, 35, 42, 26, 32, 16, 34, 33, 19, 7, 31, 36, 22, 48, 40, 25, 17, 20, 38, 12, 10, 11, 5, 2, 3, 4, 23, 1]
Custo: 1979.0
---------------------------

Custo médio:  2073.75
Tempo médio:  1.2188972249859944

Resultados salvos em: resultado_problema_2_min.txt


### Problema 3

In [220]:
## Problema 3
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_km_36, cromossomos_problema3, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_3_km.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 3\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_3_km.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 10, 12, 9, 8, 21, 22, 36, 25, 31, 17, 7, 19, 20, 33, 16, 34, 32, 26, 35, 13, 24, 29, 14, 18, 15, 28, 27, 6, 30, 4, 3, 23, 2, 5, 11, 1]
Custo: 1713.9
---------------------------

Custo médio:  1714.7450000000003
Tempo médio:  1.1531956950202584

Resultados salvos em: resultado_problema_3_km.txt


### Problema 4

In [221]:
## Problema 4
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_min_36, cromossomos_problema4, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_4_min.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 4\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_4_km.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 23, 4, 3, 2, 5, 11, 10, 12, 9, 8, 21, 30, 6, 27, 28, 15, 18, 14, 29, 24, 13, 35, 33, 16, 34, 26, 32, 19, 7, 20, 17, 25, 31, 36, 22, 1]
Custo: 1701.0
---------------------------

Custo médio:  1723.95
Tempo médio:  1.0618239250034094

Resultados salvos em: resultado_problema_4_km.txt


### Problema 5

In [222]:
## Problema 5
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_km_24, cromossomos_problema5, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_5_km.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 5\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_5_km.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 23, 2, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 5, 14, 18, 15, 6, 4, 3, 1]
Custo: 1328.6000000000001
---------------------------

Custo médio:  1328.5999999999997
Tempo médio:  0.631400770007167

Resultados salvos em: resultado_problema_5_km.txt


### Problema 6

In [223]:
## Problema 6
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_min_24, cromossomos_problema6, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_6_min.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 6\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_6_min.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 23, 2, 5, 11, 10, 12, 9, 8, 21, 22, 17, 7, 19, 16, 20, 13, 24, 14, 18, 15, 6, 4, 3, 1]
Custo: 1235.0
---------------------------

Custo médio:  1235.0
Tempo médio:  0.6359600649913772

Resultados salvos em: resultado_problema_6_min.txt


### Problema 7

In [224]:
## Problema 7
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_km_12, cromossomos_problema7, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_7_km.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 7\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_7_km.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 8, 9, 10, 12, 7, 5, 11, 2, 3, 4, 6, 1]
Custo: 672.7
---------------------------

Custo médio:  697.3100000000002
Tempo médio:  0.32234386501368134

Resultados salvos em: resultado_problema_7_km.txt


### Problema 8

In [225]:
## Problema 8
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_min_12, cromossomos_problema8, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_8_min.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 8\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_8_min.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 10, 11, 5, 7, 12, 9, 8, 2, 3, 4, 6, 1]
Custo: 606.0
---------------------------

Custo médio:  607.65
Tempo médio:  0.3185782349901274

Resultados salvos em: resultado_problema_8_min.txt


### Problema 9

### Problema 10

### Problema 11

In [226]:
## Problema 11
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_km_6, cromossomos_problema11, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_11_km.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 11\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_11_km.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 3, 6, 4, 5, 2, 1]
Custo: 344.90000000000003
---------------------------

Custo médio:  344.89999999999986
Tempo médio:  0.17621300503378734

Resultados salvos em: resultado_problema_11_km.txt


### Problema 12

In [227]:
## Problema 12
melhores = []
tempos = []
for i in range(20):
    tempo_começo = time.perf_counter()
    resultado = algoritmo_genetico(matriz_problema_min_6, cromossomos_problema12, 100, 0.4, mutacao_inversao, elitismo, cruzamento, renovacao_elitismo)
    tempo_fim = time.perf_counter()
    tempos.append(tempo_fim - tempo_começo)
    melhores.append(min(resultado, key=lambda x : x[1]))
melhores = sorted(melhores, key=lambda x: x[1])

print("Resultados encontrados:")
# for i, (rota, custo) in enumerate(melhores):
#     print(f"Top {i+1} ---------------------------")
#     print(f"Caminho: {rota}")
#     print(f"Custo: {custo}")
#     print("---------------------------\n")
print("Melhor solução geral --------")
print(f"Caminho: {melhores[0][0]}")
print(f"Custo: {melhores[0][1]}")
print("---------------------------\n")
print("Custo médio: ", sum([custo for rota, custo in melhores]) / len(melhores))
print("Tempo médio: ", sum(tempos) / len(tempos))

# Salvar também em formato texto legível
with open("resultados_genetico/resultado_problema_12_min.txt", 'w', encoding='utf-8') as f:
    f.write(f"RESULTADOS - PROBLEMA 12\n")
    
    f.write(f"CONFIGURAÇÃO:\n")
    f.write(f"  Número de execuções: 20\n")
    f.write(f"  Gerações por execução: 100\n")
    f.write(f"  Taxa de mutação: 0.4\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"ESTATÍSTICAS GERAIS\n")
    f.write(f"{'='*60}\n")
    f.write(f"Melhor caminho encontrado: {melhores[0][0]}\n")
    f.write(f"Melhor custo encontrado: {melhores[0][1]:.2f}\n")
    f.write(f"Custo médio: {(sum([custo for rota, custo in melhores]) / len(melhores)):.2f}\n")
    f.write(f"Tempo médio: {(sum(tempos) / len(tempos)):.2f}s\n")
    f.write(f"Tempo total: {sum(tempos):.2f}s\n\n")
    
    f.write(f"{'='*60}\n")
    f.write(f"TODAS SOLUÇÕES ENCONTRADAS\n")
    f.write(f"{'='*60}\n")
    
    for idx, (rota, custo) in enumerate(melhores):
        f.write(f"\nTop {idx+1} --------\n")
        f.write(f"Custo: {custo:.2f}\n")
        f.write(f"Rota: {rota}\n")
        f.write("---------------------------\n")

print(f"\nResultados salvos em: resultado_problema_12_min.txt")

Resultados encontrados:
Melhor solução geral --------
Caminho: [1, 3, 4, 6, 2, 5, 1]
Custo: 305.0
---------------------------

Custo médio:  305.0
Tempo médio:  0.1675541899865493

Resultados salvos em: resultado_problema_12_min.txt


## Memetico

In [ ]:
import random

def algoritmo_memetico(grafo, populacao_inicial, numero_iteracoes, taxa_mutacao,
                       mutacao, selecao, cruzamento, renovacao,
                       heuristicas_busca_local, probabilidade_busca_local=0.2):
    populacao_atual = populacao_inicial

    for iteracao in range(numero_iteracoes):

        n_genitores = len(populacao_atual) // 2
        genitores = selecao(populacao_atual, n_genitores)
        print(f"Numero de genitores: {len(genitores)}")
        offsprings = cruzamento(genitores, grafo)
        offsprings = mutacao(offsprings, taxa_mutacao, grafo)

        novos_offsprings = []

        for item in offsprings:
            if isinstance(item, tuple):
                individuo = item[0]
            else:
                individuo = item

            custo_individuo = calcular_custo(individuo, grafo)

            if random.random() < probabilidade_busca_local:
                melhorou = False
                for heuristica in heuristicas_busca_local:
                    nova_rota, novo_custo = busca_local_primeira_melhoria(
                        individuo, custo_individuo, heuristica, grafo
                    )

                    if novo_custo < custo_individuo:
                        individuo = nova_rota
                        custo_individuo = novo_custo
                        melhorou = True

            novos_offsprings.append((individuo, custo_individuo))

        print(f"Tamanho do offspring: {len(novos_offsprings)}")
        print(novos_offsprings)
        print(f"Tamanho da pop: {len(populacao_atual)}")

        populacao_atual = renovacao(populacao_atual, novos_offsprings, len(populacao_atual))

        melhor_custo_geracao = populacao_atual[0][1]
        print(f"Geração {iteracao + 1}/{numero_iteracoes} | Melhor Custo: {melhor_custo_geracao:.2f} | Pop: {len(populacao_atual)}")

    return populacao_atual

In [ ]:
resultado = algoritmo_memetico(
    grafo=matriz_problema_km,
    populacao_inicial=cromossomos,
    numero_iteracoes=20,
    taxa_mutacao=0.3,
    mutacao=mutacao_inversao,
    selecao=elitismo,
    cruzamento=cruzamento,
    renovacao=renovacao,
    heuristicas_busca_local=[swap, shift, inversao]
)

print(resultado)